In [4]:
import os
import json
import tempfile
import torch
from pathlib import Path
from tqdm import tqdm
from rdkit import Chem
import prolif as plf
from Bio.PDB import PDBParser, PDBIO

# --- 1. PROTEIN PREPARATION HELPER ---
def prepare_protein_for_prolif(prot_mol, target_chain="A"):
    try:
        mol_copy = Chem.Mol(prot_mol)
        try:
            mol_copy.UpdatePropertyCache(strict=False)
            Chem.SanitizeMol(mol_copy, Chem.SanitizeFlags.SANITIZE_ALL ^ Chem.SanitizeFlags.SANITIZE_PROPERTIES)
        except:
            pass
        prot_h = Chem.AddHs(mol_copy, addCoords=True)
        for atom in prot_h.GetAtoms():
            info = atom.GetPDBResidueInfo()
            if info is None:
                info = Chem.AtomPDBResidueInfo()
                info.SetResidueName("UNL")
                info.SetResidueNumber(1)
                info.SetName(f"H{atom.GetIdx()}")
            info.SetChainId(target_chain)
            atom.SetMonomerInfo(info)
        return prot_h
    except:
        return None

# --- 2. FINGERPRINT CALCULATION HELPER ---
def calculate_prolif_fingerprint(pocket_structure, ligand, fp_generator, base_name):
    """
    Calculate protein-ligand interaction fingerprint using ProLIF.
    
    Args:
        pocket_structure: BioPython structure object for the protein pocket
        ligand: RDKit molecule object for the ligand
        fp_generator: ProLIF Fingerprint object
        base_name: String identifier for extracting chain ID (e.g., '1c3b_BZB_C_362')
    
    Returns:
        Set of interaction strings in format 'RESIDUE::INTERACTION_TYPE'
    """
    try:
        # Extract Chain ID (e.g., 1c3b_BZB_C_362 -> C)
        parts = base_name.split('_')
        chain_id = parts[2] if len(parts) >= 3 else "A"
        
        with tempfile.NamedTemporaryFile(mode='w', suffix='.pdb', delete=False) as tmp_prot:
            temp_prot_path = tmp_prot.name
        
        try:
            io = PDBIO()
            io.set_structure(pocket_structure)
            io.save(temp_prot_path)
            
            prot_rdkit = Chem.MolFromPDBFile(temp_prot_path, removeHs=False, flavor=1)
            if prot_rdkit is None: 
                return set()

            protein_clean = prepare_protein_for_prolif(prot_rdkit, target_chain=chain_id)
            if protein_clean is None: 
                return set()
                
            protein_mol = plf.Molecule(protein_clean)
            ligand_with_h = Chem.AddHs(ligand, addCoords=True)
            lig_mol = plf.Molecule(ligand_with_h)
            
            # Use generate() with metadata=True to get interaction names directly
            ifp = fp_generator.generate(lig_mol, protein_mol, metadata=True)
            
            # Extract interactions from the IFP object
            active_interactions = set()
            for (lig_resid, prot_resid), interactions_dict in ifp.items():
                for interaction_name in interactions_dict.keys():
                    active_interactions.add(f"{prot_resid}::{interaction_name}")
            
            return active_interactions
            
        finally:
            if os.path.exists(temp_prot_path):
                os.remove(temp_prot_path)
    except:
        # Return the exception instance so caller can inspect the error if needed
        import sys
        return sys.exc_info()[1]

# # --- 3. THE ANALYSIS FUNCTION ---
# def run_dataset_analysis(root_path, output_name="ref_interactions.json"):
#     root = Path(root_path)
#     pocket_dir = root / "pocket_files"
#     sdf_dir = root / "sdf_files"
    
#     fp_generator = plf.Fingerprint([
#         "HBDonor", "HBAcceptor", "PiStacking", 
#         "Hydrophobic", "CationPi", "Anionic", "Cationic"
#     ])
    
#     pdb_files = list(pocket_dir.glob("*.pdb"))
#     results = {}
    
#     print(f"Found {len(pdb_files)} pockets. Starting analysis...")

#     for pdb_path in tqdm(pdb_files):
#         base_name = pdb_path.stem.replace("_pocket", "")
#         sdf_path = sdf_dir / f"{base_name}.sdf"
        
#         if not sdf_path.exists():
#             continue

#         try:
#             # 1. Load PDB
#             parser = PDBParser(QUIET=True)
#             structure = parser.get_structure(base_name, str(pdb_path))
            
#             # 2. Load SDF (Safe loading for valence errors)
#             suppl = Chem.SDMolSupplier(str(sdf_path), removeHs=False, sanitize=False)
#             ligand = suppl[0]
#             if ligand is None: continue
            
#             try:
#                 ligand.UpdatePropertyCache(strict=False)
#             except:
#                 pass # Proceed with caution

#             # 3. RUN CALCULATION (Function is now defined above!)
#             interactions = calculate_prolif_fingerprint(
#                 structure, ligand, fp_generator, base_name
#             )
            
#             results[base_name] = sorted(list(interactions))
            
#         except Exception as e:
#             print(f"  Skipping {base_name} due to unexpected error: {e}")
#             continue

#     with open(output_name, "w") as f:
#         json.dump(results, f, indent=4)
        
#     print(f"\nDone! Successfully saved {len(results)} fingerprints to {output_name}")
#     return results

# # --- 4. EXECUTION ---
# data_base_path = "/Users/sanazkazeminia/Documents/analysis_suite/data/data/AMPC_beta_lactamase/02_preprocessed"
# reference_data = run_dataset_analysis(data_base_path)

In [10]:
import os
import tempfile
from pathlib import Path
from rdkit import Chem
import prolif as plf
from Bio.PDB import PDBParser, PDBIO

# Specific file you want to test
pdb_path = Path("/Users/sanazkazeminia/Documents/analysis_suite/data/data/AMPC_beta_lactamase/02_preprocessed/pocket_files/1c3b_BZB_C_362_pocket.pdb")
sdf_dir = Path("/Users/sanazkazeminia/Documents/analysis_suite/data/data/AMPC_beta_lactamase/02_preprocessed/sdf_files")

base_name = pdb_path.stem.replace("_pocket", "")
sdf_path = sdf_dir / f"{base_name}.sdf"

print(f"Testing with: {base_name}")
print(f"Pocket: {pdb_path}")
print(f"Ligand: {sdf_path}")
print(f"Ligand exists: {sdf_path.exists()}")

if sdf_path.exists():
    # Load structures
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure(base_name, str(pdb_path))
    print("1. Loaded PDB structure")

    suppl = Chem.SDMolSupplier(str(sdf_path), removeHs=False, sanitize=False)
    ligand = suppl[0]
    print(f"2. Loaded ligand: {ligand is not None}")
    
    if ligand:
        try:
            ligand.UpdatePropertyCache(strict=False)
            Chem.SanitizeMol(ligand)
            print("3. Ligand sanitized successfully")
        except Exception as e:
            print(f"3. Ligand sanitization issue: {e}")

        fp_generator = plf.Fingerprint([
            "HBDonor", "HBAcceptor", 
            "PiStacking", "EdgeToFace", "FaceToFace",
            "Hydrophobic", 
            "CationPi", "PiCation",
            "Anionic", "Cationic",
            "MetalAcceptor", "MetalDonor",
            "XBAcceptor", "XBDonor",
        ])
        
        interactions = calculate_prolif_fingerprint(structure, ligand, fp_generator, base_name)
        
        print(f"\nReturned type: {type(interactions)}")
        print(f"Number of interactions: {len(interactions)}")
        print(f"Interactions found:\n{interactions}")

Testing with: 1c3b_BZB_C_362
Pocket: /Users/sanazkazeminia/Documents/analysis_suite/data/data/AMPC_beta_lactamase/02_preprocessed/pocket_files/1c3b_BZB_C_362_pocket.pdb
Ligand: /Users/sanazkazeminia/Documents/analysis_suite/data/data/AMPC_beta_lactamase/02_preprocessed/sdf_files/1c3b_BZB_C_362.sdf
Ligand exists: True
1. Loaded PDB structure
2. Loaded ligand: True
3. Ligand sanitized successfully

Returned type: <class 'set'>
Number of interactions: 3
Interactions found:
{'TYR221.C::Hydrophobic', 'TYR150.C::Hydrophobic', 'LEU119.C::Hydrophobic'}


[14:16:50] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
